In [ ]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from PIL import Image

import model, data, data_im2im

In [ ]:
def normalize_per_item(tensor):
    size = tensor.size()
    min_vals, _ = tensor.reshape(size[0], -1).min(1, keepdims=True)
    max_vals, _ = tensor.reshape(size[0], -1).max(1, keepdims=True)
    den = max_vals - min_vals
    
    tensor = (tensor - min_vals.unsqueeze(-1).unsqueeze(-1)) / den.unsqueeze(-1).unsqueeze(-1)
    return tensor

def tensor_to_image(tensorA, tensorB):
    tensorA = normalize_per_item(tensorA)
    tensorB = normalize_per_item(tensorB)
    tensor = torch.cat([tensorA, tensorB], dim=-1).permute(0,2,3,1)
    size = tensor.size()
    tensor = normalize_per_item(tensor).reshape(size[0] * size[1], *size[2:])
    return Image.fromarray(tensor.mul(255).numpy().astype(np.uint8))

In [ ]:
netG_en2zh = model.Generator(3,3)
_ = netG_en2zh.load_state_dict(torch.load('weights/netG_en2jp_epoch19.pth', map_location='cpu'))
_ = netG_en2zh.eval()

netG_zh2en = model.Generator(3,3)
_ = netG_zh2en.load_state_dict(torch.load('weights/netG_zh2en_epoch10.pth', map_location='cpu'))
_ = netG_zh2en.eval()

In [ ]:
loader = DataLoader(data.dataset, batch_size=52, shuffle=False)
x = next(iter(loader))

with torch.no_grad():
    y = netG_en2zh(x['en'])
    z = netG_zh2en(x['zh'])
    
y.size(), z.size()

In [ ]:
image = tensor_to_image(x['en'], y)
# image.save('./results/en2zh_weight_1e-1_batch_16_epoch_10_all.jpg')
image

In [ ]:
image = tensor_to_image(x['zh'], z)
# image.save('./results/zh2en_weight_1e-1_batch_16_epoch_10_all.jpg')
image

---

In [ ]:
netG_A2B = model.Generator(3,3)
_ = netG_A2B.load_state_dict(torch.load('weights/netG_en2jp_epoch17.pth', map_location='cpu'))
_ = netG_A2B.eval()

netG_B2A = model.Generator(3,3)
_ = netG_B2A.load_state_dict(torch.load('weights/netG_en2jp_epoch17.pth', map_location='cpu'))
_ = netG_B2A.eval()

In [ ]:
dirA = '_/_/'
dataset = data_im2im.ImagePairDataset(dirA, dirB)
dataset.SIZE = (200,200)
loader = DataLoader(dataset, batch_size=20, shuffle=True)
x = next(iter(loader))

xA, xB = x['A'], x['B']

In [ ]:
with torch.no_grad():
    y = netG_A2B(xA)
    z = netG_B2A(xB)
    
y.size(), z.size()

In [ ]:
image = tensor_to_image(xA, y)
# image.save('./results/im2im_weight_with_en2zh_weights_faces_2.jpg')
image

In [ ]:
image = tensor_to_image(xB, z)
# image.save('./results/sticker2real_weight_5e-1_batch_16_epoch_80_2.jpg')
image